In [1]:
#Libraries Used
import socket
import pickle
import pdfplumber
import time
import hashlib
import copy

In [2]:
#Closing any previous socket connection
s = socket.socket(socket.AF_INET,socket.SOCK_STREAM)
s.close()

In [3]:
#Defining host and port
host = 'local host'
port = 5000
#Defining the Socket connection
s = socket.socket(socket.AF_INET,socket.SOCK_STREAM)
s.bind(('', port)) #Binding the port and socket
s.listen(5)

In [4]:
#Loading the public keys of all clients and PKDA
#Also, Loading the private key of PKDA
with open('publicKey_registrar.pkl','rb') as file:
    publicKey_registrar = pickle.load(file)

with open('publicKey_director.pkl','rb') as file:
    publicKey_director = pickle.load(file)
    
with open('privateKey_registrar.pkl','rb') as file:
    privateKey_registrar = pickle.load(file)

with open('privateKey_director.pkl','rb') as file:
    privateKey_director = pickle.load(file)

with open('publicKey_student.pkl','rb') as file:
    publicKey_student = pickle.load(file)


public_key_dict={}
public_key_dict['registrar']= publicKey_registrar
public_key_dict['director']= publicKey_director
public_key_dict['student']= publicKey_student

In [5]:
def binary_modular_exponention(m,e,n):
    result = 1
    m = m % n
    if (m == 0):
        return 0
    while(e>0):
        if(e&1):
            result = (result*m) % n
        e = e>>1
        m = (m*m) % n
    return result

In [6]:
#Encryption Algorithm
def encrypt_data(encryption_key, message):
    cipher_text = ''
    for value in range(0, len(message)):
        encrypted_char = binary_modular_exponention(ord(message[value]), encryption_key[0], encryption_key[1])
        cipher_text += chr(encrypted_char)
    return cipher_text

In [7]:
#Decryption Algorithm
def decrypt_data(decryption_key, cipher_data):
    original_text = ''
    for value in range(0, len(cipher_data)):
        decrypted_char = binary_modular_exponention(ord(cipher_data[value]), decryption_key[0], decryption_key[1])
        original_text += chr(decrypted_char)
    return original_text

In [ ]:
while(True):
    print('Waiting for connection')
    print()
    c, addr = s.accept() #Accepting Connection
    print("CONNECTION FROM:", str(addr))
    print()
    c.send(b"Connected succcessfully to Server")
    
    #Accepting the Client Request - Request sent from Step1
    msg = c.recv(1024)
    print()
    #Step 2
    received_request = msg.decode() #Received Request from Step - 1
    print('Received message from Client : ', received_request)
    print()
    #Reading the pdf file according to requested roll no
    roll_no = received_request + ".pdf"
    
    #Step 3
    try:
        #Server converts PDF file to a text file
        pdf_file = pdfplumber.open(roll_no)
        page = pdf_file.pages[0]
        text = str(page.extract_text())
        #Appending the GMT time to the text
        gmt_time = str(time.gmtime(time.time()))
        location = "Delhi"
        grade_time = text + "||" + gmt_time + "||" + location
        print("The final Grade, Time and Location is: ",grade_time)
        pdf_file.close()
        #Hashing the grades and time using SHA256
        response = hashlib.sha256(grade_time.encode())
        print("Hashed Value: ",response.hexdigest())
        
        #Step 4
        #Double Encrypting the Response by Director and Registrar
        encrypted_text = encrypt_data(privateKey_registrar, response.hexdigest())
        double_encrypted_text = encrypt_data(privateKey_director, copy.deepcopy(encrypted_text))
        #Final Message send to Client
        final_response = grade_time + "||" + double_encrypted_text
        print('Response sent by Server(decrypted form) : ',final_response)
        
        #Encrypting final response by students public key
        final_encrypted_response = encrypt_data(publicKey_student, final_response)
        print('Response sent by Server(encrypted form) : ',final_encrypted_response)
        
        c.send(final_encrypted_response.encode())
    except Exception as e:
        c.send(b"Error Generated")
        print(e)
        print("Roll Number does not exist!")
    print()
    print('------------------Transaction end---------------------')
    c.close()

s.close()

Waiting for connection

CONNECTION FROM: ('127.0.0.1', 59482)


Received message from Client :  Grade_MT20054

The final Grade, Time and Location is:  Name: Shreya Goel
Grade: A||time.struct_time(tm_year=2021, tm_mon=11, tm_mday=17, tm_hour=19, tm_min=46, tm_sec=18, tm_wday=2, tm_yday=321, tm_isdst=0)||Delhi
Hashed Value:  80513b47849fafc4560929cbcc3a00ffc132e9d68f2ee60ccab20669c6bbd3e9
Response sent by Server(decrypted form) :  Name: Shreya Goel
Grade: A||time.struct_time(tm_year=2021, tm_mon=11, tm_mday=17, tm_hour=19, tm_min=46, tm_sec=18, tm_wday=2, tm_yday=321, tm_isdst=0)||Delhi||8qKbu78uS%G%cu6qSScbccGqq%%cKSd68%6qccGbq66Sc6bbdS
Response sent by Server(encrypted form) :  N|bIKy|b;-E
\L~I)I, ,,t;~~ww,$t)>IdL8w)>>L~,,; )~LLI,L  dt>Icb3(d3	2GGkE(GG8~\$t 

------------------Transaction end---------------------
Waiting for connection

CONNECTION FROM: ('127.0.0.1', 59642)


Received message from Client :  Grade_MT20054

The final Grade, Time and Location is